In [1]:
class TicTacToe:
    
    def __init__(self):
        self.board = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.boardx = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.boardy = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.move = 0
        
    def render(self):
        for i in range(3):
            for j in range(3):
                if self.board[3 * i + j] == 1:
                    print("X", end = "")
                elif self.board[3 * i + j] == 2:
                    print("O", end = "")
                else:
                    print(".", end = "")
            print()
        print()
        
    def reset(self):
        self.board = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.boardx = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.boardy = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.move = 0
        
    def step(self, action):
        
        if self.board[action] != 0:
            info = "Invalid move"
            
        else:
            if self.move % 2 == 0:
                self.board[action] = 1
                self.boardx[action] = 1
            else:
                self.board[action] = 2
                self.boardy[action] = 1
                
            info = None
        
        observation = self.boardx.copy() + self.boardy.copy()
        done = self.isdone()
        reward = 1
        
        self.move += 1
        
        return observation, reward, done, info
    
    def isdone(self):    
        for i in range(3):
            flag = True
            for j in range(3):
                flag = flag and (self.boardx[3 * i + j])
                
            if flag:
                return True, "X"
        
        for i in range(3):
            flag = True
            for j in range(3):
                flag = flag and (self.boardx[3 * j + i])
                
            if flag:
                return True, "X"
            
        if self.boardx[0] == 1 and self.boardx[4] == 1 and self.boardx[8] == 1:
            return True, "X"
        
        if self.boardx[2] == 1 and self.boardx[4] == 1 and self.boardx[6] == 1:
            return True, "X"
        
        for i in range(3):
            flag = True
            for j in range(3):
                flag = flag and (self.boardy[3 * i + j])
                
            if flag:
                return True, "O"
        
        for i in range(3):
            flag = True
            for j in range(3):
                flag = flag and (self.boardy[3 * j + i])
                
            if flag:
                return True, "O"
            
        if self.boardy[0] == 1 and self.boardy[4] == 1 and self.boardy[8] == 1:
            return True, "O"
        
        if self.boardy[2] == 1 and self.boardy[4] == 1 and self.boardy[6] == 1:
            return True, "O"
        
        for i in range(9):
            if self.board[i] == 0:
                return False, None
                
        return True, "Draw"
    
    def action_space(self):
        moves = []
        for i in range(9):
            if self.board[i] == 0:
                moves.append(i)
                
        return random.choice(moves)

In [2]:
import random

predTable = {}
antiPredTable = {}

def get_prediction(board, boardx, boardy):
    s = ""
    for b in board:
        s += str(b)
        
    #if s in predTable.keys():
        #return predTable[s]
    
    #else:
        #predTable[s] = model.predict([boardx + boardy])
        
        for i in range(len(board)):
            if board[i] != 0:
                predTable[s][0][i] = 0
        
        return predTable[s]
    
def get_random_action(arr):
    
    rt = 0
    for a in arr[0]:
        rt += a
        
    rt = int(rt * 1000)
    
    if rt < 2:
        rt = 2
        
    while True:
        
        r = random.randrange(1, rt) / 1000

        total = 0
        for i, a in enumerate(arr[0]):
            total += a
            if total >= r:
                return i
            
        for i, a in enumerate(arr[0]):
            if a > 0:
                return i
            
        return np.argmax(arr[0])
        
def get_anti_prediction(board, boardx, boardy):
    s = ""
    for b in board:
        s += str(b)
        
    if s in antiPredTable.keys():
        return antiPredTable[s]
    
    else:
        antiPredTable[s] = antimodel.predict([boardx + boardy])
        
        for i in range(len(board)):
            if board[i] != 0:
                antiPredTable[s][0][i] = 0
        
        return antiPredTable[s]


In [8]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.callbacks import TensorBoard
import time

LR = 1e-3
env = TicTacToe()
env.reset()
goal_steps = 15
score_requirement = 10
initial_games = 1000
move_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]


def initial_population(model = False, antimodel = False):
    training_data = []
    anti_training_data = []
    #scores = []
    #accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        anti_game_memory = []
        prev_observation = []
        
        for _ in range(goal_steps):
            
            if not model:
                action = env.action_space()
            else:
                #action = np.argmax(model.predict([env.board]))
                #predictions = model.predict([env.board])[0]
                #for i in range(9):
                    #if env.board[i] == 1:
                        #predictions[i] == 0
                    
                #action = random.choices(move_list, weights = predictions)[0]
                
                if env.move % 2 == 0:
                    prediction = get_prediction(env.board, env.boardx, env.boardy)
                    action = get_random_action(prediction)
                    
                else:
                    anti_prediction = get_anti_prediction(env.board, env.boardx, env.boardy)
                    action = get_random_action(anti_prediction)
                
            observation, reward, done, info = env.step(action)
            
            if info is not None:
                print(info)
                print(action)
                env.render()
            
            if len(prev_observation) > 0 and env.move % 2 == 1:
                game_memory.append(prev_observation)
                
            if len(prev_observation) > 0 and env.move % 2 == 0:
                anti_game_memory.append(prev_observation)
                
            if len(prev_observation) == 0:
                game_memory.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
                
            prev_observation = observation
            score += reward
            if done[0]:
                break
        
        
        if done[1] == "X":
            for data in game_memory:
                training_data.append([data, 1])
                anti_training_data.append([data, 0])
                
        if done[1] == "Draw":
            for data in game_memory:
                training_data.append([data, 0.25])
                anti_training_data.append([data, 0.75])
                
        if done[1] == "O":
            for data in game_memory:
                training_data.append([data, 0])
                anti_training_data.append([data, 1])

        env.reset()
        #scores.append(score)
    
    return training_data, anti_training_data

def neural_network_model():
    
    model = Sequential()

    model.add(Dense(32))
    model.add(Activation("relu"))
    
    model.add(Dense(64))
    model.add(Activation("relu"))

    model.add(Dense(32))
    model.add(Activation("relu"))
    
    model.add(Dense(16))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    
    #opt = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)
    
    model.compile(loss = "categorical_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

    return model

def train_model(training_data, anti_training_data, model = False, antimodel = False):

    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, 1)
    
    aX = np.array([i[0] for i in anti_training_data]).reshape(-1, len(anti_training_data[0][0]))
    ay = np.array([i[1] for i in anti_training_data]).reshape(-1, 1)
    
    print(X.shape, y.shape)
    print(aX.shape, ay.shape)
    
    #for i in range(10):
        #print(X[i], y[i])

    if not model:
        model = neural_network_model()
        antimodel = neural_network_model()
    
    model.fit(X, y, epochs = 10, batch_size = 32, validation_split = 0.2)
    antimodel.fit(aX, ay, epochs = 10, batch_size = 32, validation_split = 0.2)
    
    return model, antimodel


training_data, anti_training_data = initial_population()
random.shuffle(training_data)
random.shuffle(anti_training_data)
model, antimodel = train_model(training_data, anti_training_data)


for i in range(5):
    break
    predTable = {}
    antiPredTable = {}
    print(f"\n\nRound {i + 1}\n\n")
    training_data, anti_training_data = initial_population(model, antimodel)
    random.shuffle(training_data)
    random.shuffle(anti_training_data)
    model, antimodel = train_model(training_data, anti_training_data)


(4204, 18) (4204, 1)
(4204, 18) (4204, 1)
Epoch 1/10
106/106 [==============================] - 4s 11ms/step - loss: 0.0000e+00 - accuracy: 0.2346 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 2/10
106/106 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.2304 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 3/10
106/106 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.2304 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 4/10
106/106 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 0.2304 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 5/10
106/106 [==============================] - 1s 9ms/step - loss: 0.0000e+00 - accuracy: 0.2304 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 6/10
106/106 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.2304 - val_loss: 0.0000e+00 - val_accuracy: 0.2200
Epoch 7/10
106/106 [==============================] - 1s

In [8]:
env = TicTacToe()

win = 0
i = 1
while True:
    #env.render()
    if i % 2 == 1:
        #prediction = get_prediction(env.board, env.boardx, env.boardy)
        #print(prediction)
        #action = get_random_action(prediction)
        
        prediction = model.predict([env.boardx + env.boardy])
        for _ in range(9):
            if env.board[_] != 0:
                prediction[0][_] = 0
        
        action = np.argmax(prediction)
        
        print(f"AI played {action}")
        
    
    if i % 2 == 0:
        #action = int(input())
        action = env.action_space()
        
        
        print(f"Player played {action}")
    
    observation, reward, done, info = env.step(action)
    
    if info is not None:
        print(info)
    
    i += 1
    if done[0]:
        if done[1] == "X":
            print("AI wins")
            i = 1
            win += 1
            env.reset()
            
        elif done[1] == "Draw":
            print("Draw")
            i = 1
            win += 1
            env.reset()
            
        elif done[1] == "O":
            print("Player Wins")
            break
            
    if win > 1100:
        break
    
env.render()

print(win)

0
AI played 0
Player played 4
8
AI played 8
Player played 7
6
AI played 6
Player played 2
3
AI played 3
AI wins


KeyboardInterrupt: Interrupted by user

In [11]:
model.predict([[1,0,0,0,0,0,0,0,0] + [0,1,0,0,0,0,0,0,0]])

array([[5.7142653e-02, 1.9421999e-03, 8.9073502e-02, 1.7203173e-02,
        1.1426939e-04, 7.8061599e-01, 3.3471204e-02, 1.1821634e-03,
        1.9254841e-02]], dtype=float32)

In [4]:
predTable

{'000000000': array([[0.11415607, 0.09725049, 0.14446044, 0.08488472, 0.12015843,
         0.09932572, 0.12506065, 0.11417564, 0.1005279 ]], dtype=float32),
 '200001000': array([[0.        , 0.08065595, 0.20203237, 0.05291594, 0.33805868,
         0.        , 0.07005445, 0.04699424, 0.09756564]], dtype=float32),
 '202011000': array([[0.        , 0.46425962, 0.        , 0.42947876, 0.        ,
         0.        , 0.02274261, 0.02392691, 0.01984487]], dtype=float32),
 '212211000': array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.2870112 , 0.3085926 , 0.00798527]], dtype=float32),
 '212211120': array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.51746553]], dtype=float32),
 '000010200': array([[0.05720892, 0.06353086, 0.20854217, 0.17963073, 0.        ,
         0.05475125, 0.        , 0.22186464, 0.07956775]], dtype=float32),
 '200010210': array([[0.        , 0.45335862, 0.01789434, 

In [10]:
model.predict([[0,0,1,0,0,0,0,0,0] + [0,0,0,0,0,0,0,0,0]])

array([[0.24367677, 0.06335936, 0.05717804, 0.09484636, 0.10037541,
        0.14556794, 0.05595259, 0.04260531, 0.19643825]], dtype=float32)

In [13]:
training_data[0]

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 8]

In [13]:
#model.save("TicTac_Checkpoint_1")

INFO:tensorflow:Assets written to: TicTac_Checkpoint_1\assets


In [61]:
from sklearn.model_selection import train_test_split

X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
y = np.array([i[1] for i in training_data]).reshape(-1,)

aX = np.array([i[0] for i in anti_training_data]).reshape(-1, len(anti_training_data[0][0]))
ay = np.array([i[1] for i in anti_training_data]).reshape(-1,)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [62]:
y

array([6, 5, 0, ..., 4, 0, 8])

In [57]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

svc = SVC(kernel = "linear")
svc.fit(X_train, y_train)

svc_training_score = int(svc.score(X_train, y_train) * 100)
svc_testing_score = int(svc.score(X_test, y_test) * 100)

print(f"SVC training set accuracy: {svc_training_score} %")
print(f"SVC testing set accuracy: {svc_testing_score} %\n")

nnc = MLPClassifier(hidden_layer_sizes = (7), activation = "relu", max_iter = 10000)
nnc.fit(X_train, y_train)

nnc_training_score = int(nnc.score(X_train, y_train) * 100)
nnc_testing_score = int(nnc.score(X_test, y_test) * 100)

print(f"NNC training set accuracy: {nnc_training_score} %")
print(f"NNC testing set accuracy: {nnc_testing_score} %\n")

rfc = RandomForestClassifier(n_estimators = 50)
rfc.fit(X_train, y_train)

rfc_training_score = int(rfc.score(X_train, y_train) * 100)
rfc_testing_score = int(rfc.score(X_test, y_test) * 100)

print(f"RFC training set accuracy: {rfc_training_score} %")
print(f"RFC testing set accuracy: {rfc_testing_score} %")

SVC training set accuracy: 30 %
SVC testing set accuracy: 28 %

NNC training set accuracy: 29 %
NNC testing set accuracy: 27 %

RFC training set accuracy: 40 %
RFC testing set accuracy: 29 %


In [63]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(f"Logistic Regression accuracy: {accuracy_score(y_test, predictions) * 100 : .0f} %")

from sklearn.tree import DecisionTreeClassifier

modelD = DecisionTreeClassifier(criterion = 'entropy')
modelD.fit(X_train, y_train)

predictions = modelD.predict(X_test)
print(f"Decision Tree Classifier accuracy: {accuracy_score(y_test, predictions) * 100 : .0f} %")

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print(f"KNN accuracy: {accuracy_score(y_test, predictions) * 100 : .0f} %")

Logistic Regression accuracy:  29 %
Decision Tree Classifier accuracy:  29 %
KNN accuracy:  28 %


In [64]:
from sklearn import tree
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (4,4), dpi = 300)
tree.plot_tree(modelD,filled = True);

Error in callback <function flush_figures at 0x000001A2F026D1F0> (for post_execute):


KeyboardInterrupt: 